## Energy storage paper
This is a clean up and extension of my calculations for the subsurface energy storage paper presented in ECOS 2021. Here, I am cleaning up some of the code and add some details to the paper.

In [2]:
using DataFrames, CSV, Dates, Plots
# ]add DataFrames, CSV, Dates, Plots

## Energy balance of Denmark
The electricity supply and demand of Denmark is downloaded from the Energynet website (one of the provided links to a third party). I have checked most of it and it looks fine although it must be used with care. First we load the data as a DataFrame structure.  
Here are the links to the data webpages:  
https://www.energidataservice.dk/tso-gas/gasflow  
https://www.energidataservice.dk/gas-storage-denmark/storagecapacity   
https://www.energidataservice.dk/tso-electricity/electricitybalancenonv

In [5]:
# load the data file
file_name = "electricitybalancenonv_8_31_2021.csv"
df = DataFrame(CSV.File(file_name))
println("Data loaded successfully")

Data loaded successfully


The data file covers a period of several years which is not what I need for my calculations. Therefore, I extract shorter periods from the data:

In [6]:
# separate the relevant section
date_begin = DateTime("2020-10-01")
date_end = DateTime("2021-01-30")
d_long = df[date_begin .<= df.HourDK .<= date_end, :]
d_long.HourDK .+= Dates.Year(30)

5810-element Vector{DateTime}:
 2051-01-30T00:00:00
 2051-01-30T00:00:00
 2051-01-29T23:00:00
 2051-01-29T23:00:00
 2051-01-29T22:00:00
 2051-01-29T22:00:00
 2051-01-29T21:00:00
 2051-01-29T21:00:00
 2051-01-29T20:00:00
 2051-01-29T20:00:00
 2051-01-29T19:00:00
 2051-01-29T19:00:00
 2051-01-29T18:00:00
 ⋮
 2050-10-01T05:00:00
 2050-10-01T05:00:00
 2050-10-01T04:00:00
 2050-10-01T04:00:00
 2050-10-01T03:00:00
 2050-10-01T03:00:00
 2050-10-01T02:00:00
 2050-10-01T02:00:00
 2050-10-01T01:00:00
 2050-10-01T01:00:00
 2050-10-01T00:00:00
 2050-10-01T00:00:00

In [9]:
date_begin

2020-10-01T00:00:00

In [7]:
# extract the surplus and demand in the future
future_coef = 4.0
demand_coef = 1.5
new_elec = future_coef*(d_long.OnshoreWindPower+d_long.OffshoreWindPower)
current_demand = demand_coef*d_long.TotalLoad
surplus_elec = new_elec-current_demand
new_elec[new_elec .> current_demand] .= current_demand[new_elec .> current_demand]
surplus_elec[surplus_elec .< 0] .= 0.0
shortage_elec = current_demand-new_elec

# set the end of storage
end_storage = "2020-11-30"

"2020-11-30"

In [8]:
plot(d_long)

LoadError: Cannot convert DataFrame to series data for plotting